In [23]:
import numpy as np # for using np arrays

# for bulding and running deep learning model
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split 



def EncoderMiniBlock1(input_size, n_filters=32, dropout_prob=0.3, max_pooling=True, padding=False):
    """
    This block uses multiple convolution layers, max pool, relu activation to create an architecture for learning. 
    Dropout can be added for regularization to prevent overfitting. 
    The block returns the activation values for next layer along with a skip connection which will be used in the decoder
    """
    # Add 2 Conv Layers with relu activation and HeNormal initialization using TensorFlow 
    # Proper initialization prevents from the problem of exploding and vanishing gradients 
    # 'Same' padding will pad the input to conv layer such that the output has the same height and width (hence, is not reduced in size) 
    inputs = Input(input_size)

    conv = Conv2D(n_filters, 
                  3,   # Kernel size   
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(inputs)
    conv = Conv2D(n_filters,
                  3,   # Kernel size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(conv)
    
    # Batch Normalization will normalize the output of the last layer based on the batch's mean and standard deviation
    conv = BatchNormalization()(conv, training=False)

    # In case of overfitting, dropout will regularize the loss and gradient computation to shrink the influence of weights on output
    if dropout_prob > 0:     
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)

    # Pooling reduces the size of the image while keeping the number of channels same
    # Pooling has been kept as optional as the last encoder layer does not use pooling (hence, makes the encoder block flexible to use)
    # Below, Max pooling considers the maximum of the input slice for output computation and uses stride of 2 to traverse across input image
    if max_pooling:
        if padding==True:
            next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2),padding='same')(conv) 
        else:
            next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv) 
    else:
        next_layer = conv

    # skip connection (without max pooling) will be input to the decoder layer to prevent information loss during transpose convolutions      
    skip_connection = conv
    
    return tf.keras.Model(inputs=inputs, outputs=(next_layer, skip_connection))

In [29]:
enc1=EncoderMiniBlock((95,75,20),n_filters=10,dropout_prob=0,max_pooling=True,padding=True)
enc2=EncoderMiniBlock((48,38,20),n_filters=10,dropout_prob=0,max_pooling=True,padding=False)
enc3=EncoderMiniBlock((24,19,20),n_filters=10,dropout_prob=0,max_pooling=True,padding=True)
enc4=EncoderMiniBlock((12,10,20),n_filters=10,dropout_prob=0,max_pooling=True,padding=False)
print(enc1.summary())
print(enc2.summary())
print(enc3.summary())
print(enc4.summary())

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 95, 75, 20)]      0         
                                                                 
 conv2d_28 (Conv2D)          (None, 95, 75, 10)        1810      
                                                                 
 conv2d_29 (Conv2D)          (None, 95, 75, 10)        910       
                                                                 
 batch_normalization_7 (Bat  (None, 95, 75, 10)        40        
 chNormalization)                                                
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 48, 38, 10)        0         
 ng2D)                                                           
                                                                 
Total params: 2760 (10.78 KB)
Trainable params: 2740 (10.70

In [12]:
n_filters=10
inputs=np.zeros((1,95,75,20))
conv1=Conv2D(n_filters, 3, activation='relu', padding='same', kernel_initializer='HeNormal')(inputs)
print(conv1.shape)
conv2=Conv2D(n_filters, 3, activation='relu', padding='same', kernel_initializer='HeNormal')(conv1)
print(conv2.shape)
next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2),padding='same')(conv2)
print(next_layer.shape)

(1, 95, 75, 10)
(1, 95, 75, 10)
(1, 48, 38, 10)


In [45]:
def EncoderMiniBlock(inputs, n_filters=32, dropout_prob=0.3, max_pooling=True, padding=False):
    """
    This block uses multiple convolution layers, max pool, relu activation to create an architecture for learning. 
    Dropout can be added for regularization to prevent overfitting. 
    The block returns the activation values for next layer along with a skip connection which will be used in the decoder
    """
    # Add 2 Conv Layers with relu activation and HeNormal initialization using TensorFlow 
    # Proper initialization prevents from the problem of exploding and vanishing gradients 
    # 'Same' padding will pad the input to conv layer such that the output has the same height and width (hence, is not reduced in size) 
    conv = Conv2D(n_filters, 
                  3,   # Kernel size   
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(inputs)
    conv = Conv2D(n_filters, 
                  3,   # Kernel size
                  activation='relu',
                  padding='same',
                  kernel_initializer='HeNormal')(conv)
    
    # Batch Normalization will normalize the output of the last layer based on the batch's mean and standard deviation
    conv = BatchNormalization()(conv, training=False)

    # In case of overfitting, dropout will regularize the loss and gradient computation to shrink the influence of weights on output
    if dropout_prob > 0:     
        conv = tf.keras.layers.Dropout(dropout_prob)(conv)

    # Pooling reduces the size of the image while keeping the number of channels same
    # Pooling has been kept as optional as the last encoder layer does not use pooling (hence, makes the encoder block flexible to use)
    # Below, Max pooling considers the maximum of the input slice for output computation and uses stride of 2 to traverse across input image
    if max_pooling:
        if padding==True:
            next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2),padding='same')(conv)
        else:
            next_layer = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(conv)
    else:
        next_layer = conv

    # skip connection (without max pooling) will be input to the decoder layer to prevent information loss during transpose convolutions      
    skip_connection = conv
    
    return next_layer, skip_connection

def DecoderMiniBlock(prev_layer_input, skip_layer_input, n_filters=32, odd=False):
    """
    Decoder Block first uses transpose convolution to upscale the image to a bigger size and then,
    merges the result with skip layer results from encoder block
    Adding 2 convolutions with 'same' padding helps further increase the depth of the network for better predictions
    The function returns the decoded layer output
    """
    # Start with a transpose convolution layer to first increase the size of the image
    up = Conv2DTranspose(
                 n_filters,
                 (3,3),    # Kernel size
                 strides=(2,2),
                 padding='same')(prev_layer_input)

    # Merge the skip connection from previous block to prevent information loss
    if not odd:
        merge = concatenate([up, skip_layer_input], axis=3)
    else:
        merge = concatenate([up[:,:-1,:-1,:], skip_layer_input], axis=3)
    # Add 2 Conv Layers with relu activation and HeNormal initialization for further processing
    # The parameters for the function are similar to encoder
    conv = Conv2D(n_filters, 
                 3,     # Kernel size
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(merge)
    conv = Conv2D(n_filters,
                 3,   # Kernel size
                 activation='relu',
                 padding='same',
                 kernel_initializer='HeNormal')(conv)
    
    return conv


In [54]:
input_size=(95, 76, 20)
n_filters=8
def unet_model(input_size=(95,79,20), n_filters=[8,8,8,8,8,8,8,8,8]):
    """
    This function builds the u-net model and returns the model
    """
    inputs = Input(input_size)
    
    # Encoder includes multiple convolutional mini blocks with different maxpooling, dropout and filter parameters
    # Observe that the filters are increasing as we go deeper into the network which will increasse the # channels of the image 
    cblock1 = EncoderMiniBlock(inputs, n_filters[0],dropout_prob=0, max_pooling=True,padding=True)
    cblock2 = EncoderMiniBlock(cblock1[0],n_filters[0]*2,dropout_prob=0, max_pooling=True)
    cblock3 = EncoderMiniBlock(cblock2[0],n_filters[0]*4,dropout_prob=0.2, max_pooling=True)
    cblock4 = EncoderMiniBlock(cblock3[0],n_filters[0]*8,dropout_prob=0.2, max_pooling=True,padding=True)
    cblock5  = EncoderMiniBlock(cblock4[0], n_filters[0]*8, dropout_prob=0.3, max_pooling=False) 
    ublock6 = DecoderMiniBlock(cblock5[0], cblock4[1],  n_filters[0]*8)
    ublock7 = DecoderMiniBlock(ublock6, cblock3[1],  n_filters[0]*4)
    ublock8 = DecoderMiniBlock(ublock7, cblock2[1],  n_filters[0]*2)
    ublock9 = DecoderMiniBlock(ublock8[:,:,:,:], cblock1[1],  input_size[-1], odd=True)
    model = tf.keras.Model(inputs=inputs, outputs=ublock9)
    return model


In [55]:
model1=unet_model(n_filters=[20 for i in range(8)])

In [56]:
print(model1.summary())

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_19 (InputLayer)       [(None, 95, 79, 20)]         0         []                            
                                                                                                  
 conv2d_140 (Conv2D)         (None, 95, 79, 20)           3620      ['input_19[0][0]']            
                                                                                                  
 conv2d_141 (Conv2D)         (None, 95, 79, 20)           3620      ['conv2d_140[0][0]']          
                                                                                                  
 batch_normalization_46 (Ba  (None, 95, 79, 20)           80        ['conv2d_141[0][0]']          
 tchNormalization)                                                                         